In [ ]:
# This code started as a direct copy from opencv
# https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html

import numpy as np
import cv2 as cv

# Info about YOLO import and loading the Yolo Model
# https://docs.ultralytics.com/tasks/classify/#train

# Added this to import YOLO
from ultralytics import YOLO

# If DEBUG is True we will output all the detected signs and their confidence
DEBUG = True

# Load the YOLOv8 model, this is loading our custom trained weights for our model.
model = YOLO("small2.pt")

cap = cv.VideoCapture(0)

# Stored captured text
captured_text = []

# Stores captured confidence
captured_confidence = []

# Set a threshold for the sign to register
confidence_requirement = 0.90
# confidence_requirement = 0.30

# Counts the number of consecutive significant signs
count = 0

# Counts the number of consecutive insignificant signs
noise_count = 0

# Keeps track of the last sign
last = None

translator = [
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U",
    "V",
    "W",
    "X",
    "Y",
    "Z",
    "DEL",
    "NOTHING",
    "_",
]
live_text = ""


if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # These two lines are found here https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html
    # They reduce the size of the video

    # ret = cap.set(cv.CAP_PROP_FRAME_WIDTH, 240)
    # ret = cap.set(cv.CAP_PROP_FRAME_HEIGHT, 240)

    # Our operations on the frame come here
    # gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Display the resulting frame
    # cv.imshow('frame', gray)

    # If you scroll to the very bottom of this link
    # https://docs.ultralytics.com/modes/predict/#thread-safe-inference
    # You will find the next 3 lines of code which I took from their and applied to this similar example

    # Info about reducing output from https://github.com/ultralytics/ultralytics/issues/1896
    # Run YOLOv8 inference on the frame
    results = model(frame, verbose=False)
    top_class = results[0].probs.top1
    top_confidence = results[
        0
    ].probs.top1conf  # Get confidence of the top-class prediction

    # If the confidence of the sign is above the threshold
    if top_confidence >= confidence_requirement:

        # If the top_class is the last class
        # hence it is consecutive increase count by 1.
        if top_class == last:

            count = count + 1

        # If the top_class is not the last class
        # it is  a new class restart counter
        else:

            count = 1

        # If there are 3 consecutive significant signs track it
        if count == 3:

            if DEBUG == True:
                captured_text.append(translator[top_class])
                captured_confidence.append(top_confidence)

            if (translator[top_class] != "DEL") and (
                translator[top_class] != "NOTHING"
            ):

                live_text = live_text + translator[top_class]

            elif translator[top_class] == "DEL":

                # https://stackoverflow.com/questions/15478127/remove-final-character-from-string
                live_text = live_text[:-1]

        # Set last to be the top_class
        last = top_class

        # Set the noise counter to 0 since this is not noise
        noise_count = 0

    # If the confidence of the current sign is not enough for the threshold increase noise counter
    else:

        noise_count = noise_count + 1

    # If there are three consecutive insignificant signs
    # Reset count, allowing another consecutive sign to be registerred for instance (A,A)
    # Reset the noise counter
    if noise_count == 3:
        count = 0
        noise_count = 0

    # Visualize the results on the frame
    # annotated_frame = results[0].plot()

    # Display the annotated frame

    # Example from: https://www.geeksforgeeks.org/python-opencv-write-text-on-video/
    cv.putText(
        frame,
        "Text: " + live_text,
        (10, 460),
        cv.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 255),
        2,
        cv.LINE_4,
    )

    cv.imshow("Capture", frame)

    if cv.waitKey(1) == ord("q"):

        if DEBUG == True:

            print("\n\nCaptured Text:")

            for i in range(len(captured_text)):

                print(
                    "Translated text: ",
                    captured_text[i],
                    " Confidence: ",
                    captured_confidence[i].item(),
                )

        break


# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

In [2]:
import cv2 as cv
import tkinter as tk
from tkinter import ttk
from tkinter.scrolledtext import ScrolledText
import ttkbootstrap as ttkb  # Modern mobile-like design
from ttkbootstrap.constants import PRIMARY
from ultralytics import YOLO
from PIL import Image, ImageTk

# Load the YOLO model
model = YOLO("small2.pt")
# model = YOLO("C:/Users/mbele/Desktop/capstone/Capstone-Project/old/best_v3_yolov11.pt")
DEBUG = True
confidence_requirement = 0.90

captured_text = []
captured_confidence = []
count = 0
noise_count = 0
last = None
live_text = ""
translator = [
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U",
    "V",
    "W",
    "X",
    "Y",
    "Z",
    "DEL",
    "NOTHING",
    "_",
]

# Initialize the Tkinter window using ttkbootstrap for modern styling
window = ttkb.Window(themename="darkly")  # Set a modern, mobile-like theme
window.title("Sign Language Detection App")
window.geometry("480x800")

# Create a Label to display video feed
video_frame = ttkb.Frame(window, padding=10)
video_frame.pack(pady=10)

video_label = ttkb.Label(video_frame)
video_label.pack()

# Create a ScrolledText field to display live translated text (expandable with scroll)
translated_text_display = ScrolledText(
    window, font=("Arial", 18), height=5, wrap=tk.WORD, bg="#282828", fg="#f1f1f1"
)
translated_text_display.pack(padx=10, pady=10, fill="x")

# Function to update the video frame
cap = cv.VideoCapture(0)

# Set the desired FPS (e.g., 30 for 30 frames per second)
# FPS = 10
# frame_delay = int(1000 / FPS)  # Calculate delay in milliseconds


def update_frame():
    global last, count, noise_count, live_text

    ret, frame = cap.read()

    if ret:
        # Run YOLOv8 inference on the frame
        results = model(frame, verbose=False)
        top_class = results[0].probs.top1
        top_confidence = results[0].probs.top1conf

        if top_confidence >= confidence_requirement:
            if top_class == last:
                count += 1
            else:
                count = 1

            if count == 3:
                if DEBUG:
                    captured_text.append(translator[top_class])
                    captured_confidence.append(top_confidence)

                if translator[top_class] not in ["DEL", "NOTHING"]:
                    live_text += translator[top_class]
                elif translator[top_class] == "DEL":
                    live_text = live_text[:-1]

            last = top_class
            noise_count = 0
        else:
            noise_count += 1

        if noise_count == 3:
            count = 0
            noise_count = 0

        # Convert frame to ImageTk for Tkinter display
        img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img_tk = ImageTk.PhotoImage(image=img)

        video_label.imgtk = img_tk
        video_label.configure(image=img_tk)

        # Update the live text display (scrollable and expandable)
        translated_text_display.delete(1.0, tk.END)  # Clear current text
        translated_text_display.insert(tk.END, live_text)  # Insert updated live text
        translated_text_display.see(tk.END)  # Auto-scroll to the end

    # Continue updating the frame with the specified frame rate
    window.after(int(1), update_frame)  # Use the calculated frame delay


# Create Start and Quit buttons with modern mobile styling
def start_feed():
    update_frame()


def quit_app():
    if DEBUG:
        print("\n\nCaptured Text:")
        for i in range(len(captured_text)):
            print(
                "Translated text: ",
                captured_text[i],
                " Confidence: ",
                captured_confidence[i].item(),
            )
    cap.release()
    window.quit()


button_frame = ttkb.Frame(window, padding=10)
button_frame.pack(pady=10)

start_button = ttkb.Button(
    button_frame, text="Start Detection", command=start_feed, bootstyle=PRIMARY
)
start_button.pack(side="left", padx=10)

quit_button = ttkb.Button(
    button_frame, text="Quit", command=quit_app, bootstyle=PRIMARY
)
quit_button.pack(side="right", padx=10)

# Start the app loop
window.mainloop()

In [ ]:
# This code started as a direct copy from opencv
# https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html

import numpy as np
import cv2 as cv

# Info about YOLO import and loading the Yolo Model
# https://docs.ultralytics.com/tasks/classify/#train

# Added this to import YOLO
from ultralytics import YOLO

# If DEBUG is True we will output all the detected signs and their confidence
DEBUG = True

# Load the YOLOv8 model, this is loading our custom trained weights for our model.
model = YOLO("small2.pt")

cap = cv.VideoCapture(0)

# Stored captured text
captured_text = []

# Stores captured confidence
captured_confidence = []

# Set a threshold for the sign to register
confidence_requirement = 0.90
# confidence_requirement = 0.30

# Counts the number of consecutive significant signs
count = 0

# Counts the number of consecutive insignificant signs
noise_count = 0

# Keeps track of the last sign
last = None

translator = [
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U",
    "V",
    "W",
    "X",
    "Y",
    "Z",
    "CONFIRM",
    "DEL",
    "NOTHING",
    "_",
]
live_text = ""


if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # These two lines are found here https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html
    # They reduce the size of the video

    # ret = cap.set(cv.CAP_PROP_FRAME_WIDTH, 500)
    # ret = cap.set(cv.CAP_PROP_FRAME_HEIGHT, 500)

    # Our operations on the frame come here
    # gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Display the resulting frame
    # cv.imshow('frame', gray)

    # If you scroll to the very bottom of this link
    # https://docs.ultralytics.com/modes/predict/#thread-safe-inference
    # You will find the next 3 lines of code which I took from their and applied to this similar example

    # Info about reducing output from https://github.com/ultralytics/ultralytics/issues/1896
    # Run YOLOv8 inference on the frame
    results = model(frame, verbose=False)
    top_class = results[0].probs.top1
    top_confidence = results[
        0
    ].probs.top1conf  # Get confidence of the top-class prediction

    # If the confidence of the sign is above the threshold
    if top_confidence >= confidence_requirement:

        # If the top_class is the last class
        # hence it is consecutive increase count by 1.
        if top_class == last:

            count = count + 1

        # If the top_class is not the last class
        # it is  a new class restart counter
        else:

            count = 1

        # If there are 3 consecutive significant signs track it
        if count == 3:

            if DEBUG == True:
                captured_text.append(translator[top_class])
                captured_confidence.append(top_confidence)

            if (translator[top_class] != "DEL") and (
                translator[top_class] != "NOTHING"
            ):

                live_text = live_text + translator[top_class]

            elif translator[top_class] == "DEL":

                # https://stackoverflow.com/questions/15478127/remove-final-character-from-string
                live_text = live_text[:-1]

        # Set last to be the top_class
        last = top_class

        # Set the noise counter to 0 since this is not noise
        noise_count = 0

    # If the confidence of the current sign is not enough for the threshold increase noise counter
    else:

        noise_count = noise_count + 1

    # If there are three consecutive insignificant signs
    # Reset count, allowing another consecutive sign to be registerred for instance (A,A)
    # Reset the noise counter
    if noise_count == 3:
        count = 0
        noise_count = 0

    # Visualize the results on the frame
    # annotated_frame = results[0].plot()

    # Display the annotated frame

    # Example from: https://www.geeksforgeeks.org/python-opencv-write-text-on-video/
    cv.putText(
        frame,
        "Text: " + live_text,
        (10, 460),
        cv.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 255),
        2,
        cv.LINE_4,
    )

    cv.imshow("Capture", frame)

    if cv.waitKey(1) == ord("q"):

        if DEBUG == True:

            print("\n\nCaptured Text:")

            for i in range(len(captured_text)):

                print(
                    "Translated text: ",
                    captured_text[i],
                    " Confidence: ",
                    captured_confidence[i].item(),
                )

        break


# When everything done, release the capture
cap.release()
cv.destroyAllWindows()



Captured Text:


In [ ]:
# Set the desired FPS (e.g., 30 for 30 frames per second)
FPS = 10
frame_delay = int(1000 / FPS)  # Calculate delay in milliseconds


def update_frame():
    global last, count, noise_count, live_text

    ret, frame = cap.read()

    if ret:
        # Run YOLOv8 inference on the frame
        results = model(frame, verbose=False)
        top_class = results[0].probs.top1
        top_confidence = results[0].probs.top1conf

        if top_confidence >= confidence_requirement:
            if top_class == last:
                count += 1
            else:
                count = 1

            if count == 3:
                if DEBUG:
                    captured_text.append(translator[top_class])
                    captured_confidence.append(top_confidence)

                if translator[top_class] not in ["DEL", "NOTHING"]:
                    live_text += translator[top_class]
                elif translator[top_class] == "DEL":
                    live_text = live_text[:-1]

            last = top_class
            noise_count = 0
        else:
            noise_count += 1

        if noise_count == 3:
            count = 0
            noise_count = 0

        # Convert frame to ImageTk for Tkinter display
        img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img_tk = ImageTk.PhotoImage(image=img)

        video_label.imgtk = img_tk
        video_label.configure(image=img_tk)

        # Update the live text display (scrollable and expandable)
        translated_text_display.delete(1.0, tk.END)  # Clear current text
        translated_text_display.insert(tk.END, live_text)  # Insert updated live text
        translated_text_display.see(tk.END)  # Auto-scroll to the end

    # Continue updating the frame with the specified frame rate
    window.after(frame_delay, update_frame)  # Use the calculated frame delay